In [5]:
#%run 'word2vec_data.ipynb'
import cPickle as pickle
%pylab inline

import scipy
import numpy as np
import pandas as pd
#import plotly.plotly as py
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords

from gensim import models


import math
import codecs
import re
import string
from time import time


## Sklearn
from sklearn import svm, datasets, preprocessing, metrics
import sklearn

from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score, roc_auc_score, accuracy_score
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score, precision_score
from sklearn.utils import shuffle


from sklearn.svm import SVC
from sklearn import datasets, feature_selection, cluster, feature_extraction, decomposition, metrics, model_selection
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split, cross_val_score

from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.svm import LinearSVC
from scipy import interp

#############################

#############################
### other stuff
from IPython import display

import os
#import ipyparallel as ipp
import warnings
warnings.filterwarnings('ignore')

# This is here to avoid all the Matplotlib warnings due to current bugs 
# - not a good idea to keep around
print ("Required Libraries loaded.")

Populating the interactive namespace from numpy and matplotlib
Required Libraries loaded.


In [6]:
#!pip install --upgrade version_information
#%reload_ext version_information
%load_ext version_information 
%version_information scipy, numpy, sklearn, pandas, scipy, nltk, gensim, matplotlib

The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information


Software,Version
Python,2.7.13 64bit [MSC v.1500 64 bit (AMD64)]
IPython,5.1.0
OS,Windows 7 6.1.7601 SP1
scipy,0.18.1
numpy,1.11.3
sklearn,0.18.1
pandas,0.19.2
scipy,0.18.1
nltk,3.2.2
gensim,1.0.1


In [7]:
inhibitor  = pd.read_csv("C:/EPC_Data/complete_data/raw/ACEInhibitors.csv", sep=",", index_col = "PMID")

In [8]:
seed = 29
# shuffle dataset and split to train and test
inhibitor = shuffle(inhibitor, random_state = seed)
inhibitor.head(2)

,study,EndNote_Id,init_screen,fin_screen,Abstract,MeSH,Mh_MeSH,Pt_Pub_Type,Pub_Type,TiAbs,TiAbsMeSHPT,TiAbsMeSHPT_append,TiAbsMeSH_append,TiAbsMesh,Title,Label
PMID,,,,,,,,,,,,,,,,
12067927,ACEInhibitors,7961,E,E,NaN,Adult Aged Coronary Disease Female Hospitaliz...,"[MHAdult, MHAged, MHCoronary, MHDisease, MHFem...","[PTComment, PTEditorial]",Comment Editorial,Patients with acute coronary syndrome should s...,Patients with acute coronary syndrome should s...,Patients with acute coronary syndrome should s...,Patients with acute coronary syndrome should s...,Patients with acute coronary syndrome should s...,Patients with acute coronary syndrome should s...,0
12181216,ACEInhibitors,8138,E,E,To determine the clinical and prognostic diff...,Adult Aged Female Heart Failure Hospitalizati...,"[MHAdult, MHAged, MHFemale, MHHeart, MHFailure...","[PTJournal, PTArticle]",Journal Article,Clinical characteristics and prognosis of hosp...,Clinical characteristics and prognosis of hosp...,Clinical characteristics and prognosis of hosp...,Clinical characteristics and prognosis of hosp...,Clinical characteristics and prognosis of hosp...,Clinical characteristics and prognosis of hosp...,0


In [9]:
folder = "C:/EPC_Data/complete_data/word2vec"
filepath = os.path.join(folder, "ACEInhibitors_chi2_features_10minwords_15context")
model = models.Word2Vec.load(filepath)
print ("\nMatrix of word vectors is size : ", model.wv.syn0.shape)

('\nMatrix of word vectors is size : ', (3441L, 210L))


In [10]:
#Create feature vectors from the averages of word vectors

def makeFeatureVec(words, model):
    """
    words - list of words (i.e. article) to be used as input for the creation of word vectors
    model - model to use for the creation of the vectors
    
    makeFeatureVec: Function to average all of the word vectors in a given paragraph
    returns: a numpy array of floats that are the average of the constituent word vectors for each word
    """
    num_features = model.wv.syn0.shape[1]
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the article and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec

def getAvgFeatureVecs(articles, model):
    """
    articles - list of articles for which the creation of word vectors needs to be done for each
    model - model to use for the creation of the vectors
    
    getAvgFeatureVecs: Given a set of articles (each one a list of words), calculate 
    the average feature vector for each one and return a 2D numpy array 
    returns: a 2D numpy array that contains the average of the constituent word vectors for each article
    """
    num_features = model.wv.syn0.shape[1]
    articleFeatureVecs = np.zeros((len(articles),num_features),dtype="float32")
    counter = 0.
    
    # Loop through the articles
    for article in articles:
        #
        # Print a status message every 1000th review
        if counter%500. == 0.:
            print ("Article %d of %d" % (counter, len(articles)))
        # 
        # Call the function (defined above) that makes average feature vectors
        articleFeatureVecs[counter] = makeFeatureVec(article, model)
        #
        # Increment the counter
        counter = counter + 1.
    return articleFeatureVecs

In [11]:
# Import articles for string cleaning

def clean_text(text, stem_words = False, remove_stopwords = False):
    ###clean
    texts = re.sub("[^a-zA-Z]",    #pattern to match
              " ",              #replace other with this
              str(text))                 #text to apply to
         
    #print "Text recived: ", texts 
    clean_corpus = texts.lower().split()
    #print "corpus: ", clean_corpus
    if stem_words:
        # Porter stemmer
        porter = nltk.PorterStemmer()
        # Snowball stemmer
        snowball = nltk.SnowballStemmer('english')
        # Lancaster stemmer
        lancaster = nltk.LancasterStemmer()
        # General stemming Lambda function to stem tokens
        clean_corpus = lambda tokens: [porter.stem(w) for w in corpus]
    if remove_stopwords:   # Optionally remove stop words
        stops = set(stopwords.words("english"))
        clean_corpus = [w for w in clean_corpus if not w in stops]
        #print "Clean_corpus: ", clean_corpus
    
    return (clean_corpus)

In [12]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. We remove stopwords.

print ("Creating average feature vecs for articles")
cleaned_articles = []
for article in inhibitor.TiAbsMesh:
    cleaned_articles.append(clean_text(article, stem_words=False, remove_stopwords=True ))

averageWordVecs = getAvgFeatureVecs(cleaned_articles, model)

Creating average feature vecs for articles
Article 0 of 2544
Article 500 of 2544
Article 1000 of 2544
Article 1500 of 2544
Article 2000 of 2544
Article 2500 of 2544


In [13]:
RANDOM_STATE = 55
TEST_SIZE = 0.3

X_train, X_test, y_train, y_test = train_test_split(averageWordVecs, inhibitor.Label, test_size=TEST_SIZE, 
                                                    random_state=37)

print (len(X_train))
print (len(X_test))

1780
764


In [14]:
X = averageWordVecs
y = inhibitor.Label
le = preprocessing.LabelEncoder()
y  = le.fit_transform(y)
yTrFreq = scipy.stats.itemfreq(inhibitor.Label)
print(yTrFreq)

[[   0 2503]
 [   1   41]]


In [15]:
def weight(x):
    x = np.where(x==0, 1, 4)
    return x 

In [16]:
# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf', "sigmoid"], 'gamma': [1e-3, 1e-4],
                     'class_weight':['balanced', None],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000], 'class_weight':['balanced', None]}]

scores = ['recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(), tuned_parameters, cv=2, scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(metrics.classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for recall
()
Best parameters set found on development set:
()
{'kernel': 'rbf', 'C': 1000, 'gamma': 0.001, 'class_weight': 'balanced'}
()
Grid scores on development set:
()
0.500 (+/-0.000) for {'kernel': 'rbf', 'C': 1, 'gamma': 0.001, 'class_weight': 'balanced'}
0.500 (+/-0.000) for {'kernel': 'sigmoid', 'C': 1, 'gamma': 0.001, 'class_weight': 'balanced'}
0.500 (+/-0.000) for {'kernel': 'rbf', 'C': 1, 'gamma': 0.0001, 'class_weight': 'balanced'}
0.500 (+/-0.000) for {'kernel': 'sigmoid', 'C': 1, 'gamma': 0.0001, 'class_weight': 'balanced'}
0.500 (+/-0.000) for {'kernel': 'rbf', 'C': 1, 'gamma': 0.001, 'class_weight': None}
0.500 (+/-0.000) for {'kernel': 'sigmoid', 'C': 1, 'gamma': 0.001, 'class_weight': None}
0.500 (+/-0.000) for {'kernel': 'rbf', 'C': 1, 'gamma': 0.0001, 'class_weight': None}
0.500 (+/-0.000) for {'kernel': 'sigmoid', 'C': 1, 'gamma': 0.0001, 'class_weight': None}
0.508 (+/-0.006) for {'kernel': 'rbf', 'C': 10, 'gamma': 0.001, 'class_weigh

In [17]:
clf = SVC(C=1000.0, cache_size=200, class_weight='balanced', coef0=0.0,decision_function_shape=None, degree=3, 
          gamma='auto', kernel='sigmoid', max_iter=-1, probability=False, random_state=55, shrinking=True,
          tol=0.001, verbose=False)

seeds = [35, 71, 21, 61, 55]

svm_precision = []
svm_recall = []
svm_f = []
acc = []
ps_vectors = []
ns_vectors = []
tr_pos = []
tr_neg = []
te_pos = []
te_neg = []
true_pos = []
true_neg = []
false_pos = []
false_neg = []


for i in range(0, 5):
    
    seed = seeds[i]   
    skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)

    for train_index, test_index in skf.split(X, y):
        
           
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        w = weight(y_train)

        yTrFreq = scipy.stats.itemfreq(y_train)
        tr_pos = np.append(tr_pos, yTrFreq[1][1])
        tr_neg = np.append(tr_neg, yTrFreq[0][1])

        # print(yTrFreq)

        yTrFreq = scipy.stats.itemfreq(y_test)
        te_pos = np.append(te_pos, yTrFreq[1][1])
        te_neg = np.append(te_neg, yTrFreq[0][1])

        #print(yTrFreq)

        print ("Fitting for Seed = %d" % seed)  

        clf.fit(X_train, y_train, sample_weight = w)

        pred = clf.predict(X_test)

        s_vector = clf.n_support_


        s_prec = metrics.precision_score(y_test, pred)#, sample_weight=wt)
        s_recall = metrics.recall_score(y_test, pred)#, sample_weight=wt)
        s_f1 = metrics.f1_score(y_test, pred)#, sample_weight=wt)

        print (metrics.confusion_matrix(y_test, pred))
        true_neg.append(metrics.confusion_matrix(y_test, pred)[0,0])
        true_pos.append(metrics.confusion_matrix(y_test, pred)[1,1])
        false_neg.append(metrics.confusion_matrix(y_test, pred)[1, 0])
        false_pos.append(metrics.confusion_matrix(y_test, pred)[0, 1])

        acc_s = metrics.accuracy_score(y_test, pred)
        svm_precision.append(s_prec)
        svm_recall.append(s_recall)
        svm_f.append(s_f1)
        acc.append(acc_s)
        ps_vectors = np.append(ps_vectors, s_vector[1])
        ns_vectors = np.append(ns_vectors, s_vector[0])      

print ("SVM Cross validation results: ")
print ("Mean Precision: %.2f +/- %.2f "%(np.mean(svm_precision), np.std(svm_precision)))
print ("  Mean  Recall: %.2f +/- %.2f" %(np.mean(svm_recall), np.std(svm_recall)))
print ("      Mean  F1: %.2f +/- %.2f" %(np.mean(svm_f), np.std(svm_f)))
print ("Mean  Accuracy: %.2f +/- %.2f" %(np.mean(acc), np.std(acc)))
print (" True Negative: ", true_neg)
print ("False Negative: ", false_neg)
print (" True Positive: ", true_pos)
print ("False Positive: ", false_pos)
print ("Negative support Vectors: ", ns_vectors)
print ("Positive support Vectors: ", ps_vectors)
print ("Mean positive support vectors: %.2f +/- %.2f"%(np.mean(ps_vectors), np.std(ps_vectors)))
print ("Mean Negative support vectors: %.2f +/- %.2f"%(np.mean(ns_vectors), np.std(ns_vectors)))
print ("Train positive: %.2f" %(np.mean(tr_pos)))
print ("Train negattive: %.2f" %(np.mean(tr_neg)))
print ("Test positive: %.2f" %(np.mean(te_pos)))
print ("Test negative: %.2f" %(np.mean(te_neg)))

Fitting for Seed = 35
[[1112  140]
 [   2   19]]
Fitting for Seed = 35
[[1046  205]
 [   3   17]]
Fitting for Seed = 71
[[1107  145]
 [   3   18]]
Fitting for Seed = 71
[[1069  182]
 [   5   15]]
Fitting for Seed = 21
[[1146  106]
 [   8   13]]
Fitting for Seed = 21
[[1014  237]
 [   2   18]]
Fitting for Seed = 61
[[1056  196]
 [   2   19]]
Fitting for Seed = 61
[[1098  153]
 [   1   19]]
Fitting for Seed = 55
[[1132  120]
 [   1   20]]
Fitting for Seed = 55
[[975 276]
 [  2  18]]
SVM Cross validation results: 
Mean Precision: 0.10 +/- 0.02 
  Mean  Recall: 0.86 +/- 0.10
      Mean  F1: 0.17 +/- 0.04
Mean  Accuracy: 0.86 +/- 0.04
(' True Negative: ', [1112, 1046, 1107, 1069, 1146, 1014, 1056, 1098, 1132, 975])
('False Negative: ', [2, 3, 3, 5, 8, 2, 2, 1, 1, 2])
(' True Positive: ', [19, 17, 18, 15, 13, 18, 19, 19, 20, 18])
('False Positive: ', [140, 205, 145, 182, 106, 237, 196, 153, 120, 276])
('Negative support Vectors: ', array([ 291.,  443.,  323.,  397.,  265.,  492.,  487.,  335

In [18]:
clf = SVC(C=1000.0, cache_size=200, class_weight='balanced', coef0=0.0,decision_function_shape=None, degree=3, 
          gamma='auto', kernel='rbf', max_iter=-1, probability=False, random_state=55, shrinking=True,
          tol=0.001, verbose=False)

seeds = [35, 71, 21, 61, 55]

svm_precision = []
svm_recall = []
svm_f = []
acc = []
ps_vectors = []
ns_vectors = []
tr_pos = []
tr_neg = []
te_pos = []
te_neg = []
true_pos = []
true_neg = []
false_pos = []
false_neg = []


for i in range(0, 5):
    
    seed = seeds[i]   
    skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)

    for train_index, test_index in skf.split(X, y):
        
           
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        w = weight(y_train)

        yTrFreq = scipy.stats.itemfreq(y_train)
        tr_pos = np.append(tr_pos, yTrFreq[1][1])
        tr_neg = np.append(tr_neg, yTrFreq[0][1])

        # print(yTrFreq)

        yTrFreq = scipy.stats.itemfreq(y_test)
        te_pos = np.append(te_pos, yTrFreq[1][1])
        te_neg = np.append(te_neg, yTrFreq[0][1])

        #print(yTrFreq)

        print ("Fitting for Seed = %d" % seed)  

        clf.fit(X_train, y_train, sample_weight = w)

        pred = clf.predict(X_test)

        s_vector = clf.n_support_


        s_prec = metrics.precision_score(y_test, pred)#, sample_weight=wt)
        s_recall = metrics.recall_score(y_test, pred)#, sample_weight=wt)
        s_f1 = metrics.f1_score(y_test, pred)#, sample_weight=wt)

        print (metrics.confusion_matrix(y_test, pred))
        true_neg.append(metrics.confusion_matrix(y_test, pred)[0,0])
        true_pos.append(metrics.confusion_matrix(y_test, pred)[1,1])
        false_neg.append(metrics.confusion_matrix(y_test, pred)[1, 0])
        false_pos.append(metrics.confusion_matrix(y_test, pred)[0, 1])

        acc_s = metrics.accuracy_score(y_test, pred)
        svm_precision.append(s_prec)
        svm_recall.append(s_recall)
        svm_f.append(s_f1)
        acc.append(acc_s)
        ps_vectors = np.append(ps_vectors, s_vector[1])
        ns_vectors = np.append(ns_vectors, s_vector[0])  
print ("SVM Cross validation results: ")
print ("Mean Precision: %.2f +/- %.2f "%(np.mean(svm_precision), np.std(svm_precision)))
print ("  Mean  Recall: %.2f +/- %.2f" %(np.mean(svm_recall), np.std(svm_recall)))
print ("      Mean  F1: %.2f +/- %.2f" %(np.mean(svm_f), np.std(svm_f)))
print ("Mean  Accuracy: %.2f +/- %.2f" %(np.mean(acc), np.std(acc)))
print (" True Negative: ", true_neg)
print ("False Negative: ", false_neg)
print (" True Positive: ", true_pos)
print ("False Positive: ", false_pos)
print ("Negative support Vectors: ", ns_vectors)
print ("Positive support Vectors: ", ps_vectors)
print ("Mean positive support vectors: %.2f +/- %.2f"%(np.mean(ps_vectors), np.std(ps_vectors)))
print ("Mean Negative support vectors: %.2f +/- %.2f"%(np.mean(ns_vectors), np.std(ns_vectors)))
print ("Train positive: %.2f" %(np.mean(tr_pos)))
print ("Train negattive: %.2f" %(np.mean(tr_neg)))
print ("Test positive: %.2f" %(np.mean(te_pos)))
print ("Test negative: %.2f" %(np.mean(te_neg)))

Fitting for Seed = 35
[[1132  120]
 [   3   18]]
Fitting for Seed = 35
[[1086  165]
 [   3   17]]
Fitting for Seed = 71
[[1135  117]
 [   4   17]]
Fitting for Seed = 71
[[1105  146]
 [   7   13]]
Fitting for Seed = 21
[[1159   93]
 [   8   13]]
Fitting for Seed = 21
[[1063  188]
 [   2   18]]
Fitting for Seed = 61
[[1117  135]
 [   4   17]]
Fitting for Seed = 61
[[1111  140]
 [   1   19]]
Fitting for Seed = 55
[[1156   96]
 [   3   18]]
Fitting for Seed = 55
[[1029  222]
 [   1   19]]
SVM Cross validation results: 
Mean Precision: 0.11 +/- 0.02 
  Mean  Recall: 0.83 +/- 0.11
      Mean  F1: 0.19 +/- 0.04
Mean  Accuracy: 0.89 +/- 0.03
(' True Negative: ', [1132, 1086, 1135, 1105, 1159, 1063, 1117, 1111, 1156, 1029])
('False Negative: ', [3, 3, 4, 7, 8, 2, 4, 1, 3, 1])
(' True Positive: ', [18, 17, 17, 13, 13, 18, 17, 19, 18, 19])
('False Positive: ', [120, 165, 117, 146, 93, 188, 135, 140, 96, 222])
('Negative support Vectors: ', array([ 231.,  362.,  268.,  321.,  209.,  400.,  399.,  

In [19]:
clf = SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,decision_function_shape=None, degree=3, 
          gamma='auto', kernel='linear', max_iter=-1, probability=False, random_state=55, shrinking=True,
          tol=0.001, verbose=False)

seeds = [35, 71, 21, 61, 55]

svm_precision = []
svm_recall = []
svm_f = []
acc = []
ps_vectors = []
ns_vectors = []
tr_pos = []
tr_neg = []
te_pos = []
te_neg = []
true_pos = []
true_neg = []
false_pos = []
false_neg = []


for i in range(0, 5):
    
    seed = seeds[i]   
    skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)

    for train_index, test_index in skf.split(X, y):
        
           
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        w = weight(y_train)

        yTrFreq = scipy.stats.itemfreq(y_train)
        tr_pos = np.append(tr_pos, yTrFreq[1][1])
        tr_neg = np.append(tr_neg, yTrFreq[0][1])

        # print(yTrFreq)

        yTrFreq = scipy.stats.itemfreq(y_test)
        te_pos = np.append(te_pos, yTrFreq[1][1])
        te_neg = np.append(te_neg, yTrFreq[0][1])

        #print(yTrFreq)

        print ("Fitting for Seed = %d" % seed)  

        clf.fit(X_train, y_train, sample_weight = w)

        pred = clf.predict(X_test)

        s_vector = clf.n_support_


        s_prec = metrics.precision_score(y_test, pred)#, sample_weight=wt)
        s_recall = metrics.recall_score(y_test, pred)#, sample_weight=wt)
        s_f1 = metrics.f1_score(y_test, pred)#, sample_weight=wt)

        print (metrics.confusion_matrix(y_test, pred))
        true_neg.append(metrics.confusion_matrix(y_test, pred)[0,0])
        true_pos.append(metrics.confusion_matrix(y_test, pred)[1,1])
        false_neg.append(metrics.confusion_matrix(y_test, pred)[1, 0])
        false_pos.append(metrics.confusion_matrix(y_test, pred)[0, 1])

        acc_s = metrics.accuracy_score(y_test, pred)
        svm_precision.append(s_prec)
        svm_recall.append(s_recall)
        svm_f.append(s_f1)
        acc.append(acc_s)
        ps_vectors = np.append(ps_vectors, s_vector[1])
        ns_vectors = np.append(ns_vectors, s_vector[0])    

print ("SVM Cross validation results: ")
print ("Mean Precision: %.2f +/- %.2f "%(np.mean(svm_precision), np.std(svm_precision)))
print ("  Mean  Recall: %.2f +/- %.2f" %(np.mean(svm_recall), np.std(svm_recall)))
print ("      Mean  F1: %.2f +/- %.2f" %(np.mean(svm_f), np.std(svm_f)))
print ("Mean  Accuracy: %.2f +/- %.2f" %(np.mean(acc), np.std(acc)))
print (" True Negative: ", true_neg)
print ("False Negative: ", false_neg)
print (" True Positive: ", true_pos)
print ("False Positive: ", false_pos)
print ("Negative support Vectors: ", ns_vectors)
print ("Positive support Vectors: ", ps_vectors)
print ("Mean positive support vectors: %.2f +/- %.2f"%(np.mean(ps_vectors), np.std(ps_vectors)))
print ("Mean Negative support vectors: %.2f +/- %.2f"%(np.mean(ns_vectors), np.std(ns_vectors)))
print ("Train positive: %.2f" %(np.mean(tr_pos)))
print ("Train negattive: %.2f" %(np.mean(tr_neg)))
print ("Test positive: %.2f" %(np.mean(te_pos)))
print ("Test negative: %.2f" %(np.mean(te_neg)))

Fitting for Seed = 35
[[1029  223]
 [   1   20]]
Fitting for Seed = 35
[[901 350]
 [  2  18]]
Fitting for Seed = 71
[[1033  219]
 [   1   20]]
Fitting for Seed = 71
[[945 306]
 [  3  17]]
Fitting for Seed = 21
[[1075  177]
 [   2   19]]
Fitting for Seed = 21
[[909 342]
 [  0  20]]
Fitting for Seed = 61
[[938 314]
 [  0  21]]
Fitting for Seed = 61
[[1011  240]
 [   1   19]]
Fitting for Seed = 55
[[1044  208]
 [   1   20]]
Fitting for Seed = 55
[[882 369]
 [  1  19]]
SVM Cross validation results: 
Mean Precision: 0.07 +/- 0.02 
  Mean  Recall: 0.94 +/- 0.04
      Mean  F1: 0.13 +/- 0.03
Mean  Accuracy: 0.78 +/- 0.05
(' True Negative: ', [1029, 901, 1033, 945, 1075, 909, 938, 1011, 1044, 882])
('False Negative: ', [1, 2, 1, 3, 2, 0, 0, 1, 1, 1])
(' True Positive: ', [20, 18, 20, 17, 19, 20, 21, 19, 20, 19])
('False Positive: ', [223, 350, 219, 306, 177, 342, 314, 240, 208, 369])
('Negative support Vectors: ', array([ 508.,  663.,  526.,  622.,  468.,  720.,  702.,  551.,  507.,  683.]))
(